# Error handling

ironArray has a sophisticated error handling machinery allowing to catch errors at multiple levels. To see how all this works, let's try to open an on disk iarray  which does not exist.

In [1]:
import iarray as ia

In [2]:
a = ia.open("non_ondisk_array.iarr")

IArrayError: b'FILE OPEN - 0x800a000000510008 - error=1,ver=0,rev=10,os=0,neg=0,adj=81,subject=8,code=5308416,ubits=0x0'

The ironArray error handling machinery raised a Python error called ``IArrayError``. This error returns a general error message. In this particular case, the error is ``FILE OPEN`` but it does not indicate much more beyond this.

In order to obtain more detailed information, we can set the environment variable ``INAC_TRACE="iarray.error"``. With this, ironArray will show us a more detailed error; in particular, it will provide us with a trace of what's going on in the underlying C library.

To better leverage this functionality inside a Jupyter notebook, it is convenient to use the ``wurlitzer`` extension to redirect `stdout` and `stderr` handlers to the notebook:

In [ ]:
%load_ext wurlitzer

import os
os.environ['INAC_TRACE'] = 'iarray.error'

and, after doing this, we can get more info on the ironArray core library (written in C):

In [ ]:
a = ia.open("non_ondisk_array.iarr")

Now, we get the error message ``File not exists`` that already gives us a better idea of what we were doing wrong.

In addition, we can see in which lines of the ironArray core code the error has
occurred. In this case, the trace is:

- ``iarray/iarray-c-develop/src/iarray_container.c:120``

While those will be mostly useful for developers, it is always interesting that you can attach these  when you file possible bug reports.

That's all.  Whenever you would like more info on your errors, remember to activate the `INAC_TRACE` environment variable.